# First things first

The National Agency of Petroleum, Natural Gas and Bio fuels (ANP in Portuguese) releases weekly reports of gas, diesel and other fuels prices used in transportation across the country. 

https://www.kaggle.com/matheusfreitag/gas-prices-in-brazil/data

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
data = pd.read_csv('2004-2019.tsv', sep = '\t', index_col = 0)
data.sample(3) # Let's take a look

,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO,MÊS,ANO
3110,2006-08-06,2006-08-12,NORDESTE,PIAUI,ETANOL HIDRATADO,102,R$/l,2.204,0.086,2.120,2.40,0.342,0.039,1.862,0.046,1.75,1.9482,0.025,8,2006
49091,2008-04-27,2008-05-03,NORDESTE,PIAUI,ÓLEO DIESEL,101,R$/l,1.906,0.045,1.810,2.05,0.159,0.024,1.747,0.03,1.67,1.8256,0.017,4,2008
39102,2007-10-28,2007-11-03,SUDESTE,ESPIRITO SANTO,GNV,13,R$/m3,1.439,0.147,1.438,1.89,0.596,0.102,0.843,0.382,0.7686,1.62,0.453,10,2007


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106823 entries, 0 to 106822
Data columns (total 20 columns):
DATA INICIAL                     106823 non-null object
DATA FINAL                       106823 non-null object
REGIÃO                           106823 non-null object
ESTADO                           106823 non-null object
PRODUTO                          106823 non-null object
NÚMERO DE POSTOS PESQUISADOS     106823 non-null int64
UNIDADE DE MEDIDA                106823 non-null object
PREÇO MÉDIO REVENDA              106823 non-null float64
DESVIO PADRÃO REVENDA            106823 non-null float64
PREÇO MÍNIMO REVENDA             106823 non-null float64
PREÇO MÁXIMO REVENDA             106823 non-null float64
MARGEM MÉDIA REVENDA             106823 non-null object
COEF DE VARIAÇÃO REVENDA         106823 non-null float64
PREÇO MÉDIO DISTRIBUIÇÃO         106823 non-null object
DESVIO PADRÃO DISTRIBUIÇÃO       106823 non-null object
PREÇO MÍNIMO DISTRIBUIÇÃO        106823 non-n

In [4]:
data.PRODUTO.value_counts() # which are the products?

ÓLEO DIESEL         21194
GASOLINA COMUM      21194
GLP                 21186
ETANOL HIDRATADO    21102
GNV                 13034
ÓLEO DIESEL S10      9113
Name: PRODUTO, dtype: int64

## Diesel prices modeling

In [5]:
diesel = data[data.PRODUTO == 'ÓLEO DIESEL'].copy()

formating date columns (weekly instances from data.info())

In [6]:
diesel['DATA INICIAL'] = pd.to_datetime(diesel['DATA INICIAL']) # Date columns are str instead of DateTime
diesel['DATA FINAL'] = pd.to_datetime(diesel['DATA FINAL']) # Date columns are str instead of DateTime

Time for sanity tests

In [7]:
print((diesel['DATA FINAL'] >= diesel['DATA INICIAL']).value_counts()) # final date older than initial
print(diesel['DATA FINAL'].dt.weekday.unique(), diesel['DATA INICIAL'].dt.weekday.unique())
print('DATA INICIAL \n', diesel['DATA INICIAL'].describe())
print('DATA FINAL \n', diesel['DATA FINAL'].describe())

True    21194
dtype: int64
[5] [6]
DATA INICIAL 
 count                   21194
unique                    785
top       2015-05-24 00:00:00
freq                       27
first     2004-05-09 00:00:00
last      2019-06-23 00:00:00
Name: DATA INICIAL, dtype: object
DATA FINAL 
 count                   21194
unique                    785
top       2013-02-09 00:00:00
freq                       27
first     2004-05-15 00:00:00
last      2019-06-29 00:00:00
Name: DATA FINAL, dtype: object


Dataset splits

In [8]:
diesel_train = diesel[diesel['DATA FINAL'] < '2011-05-15'] # 7 years 
diesel_val   = diesel[(diesel['DATA FINAL'] >= '2011-05-15') & (diesel['DATA FINAL'] < '2018-01-01')] # 7 years
diesel_test  = diesel[diesel['DATA FINAL'] >= '2018-05-15'] # 1 year
diesel_train.shape, diesel_val.shape, diesel_test.shape 

((9773, 20), (9315, 20), (1593, 20))

**Target definition**

In [9]:
y_train = diesel_train['PREÇO MÉDIO REVENDA']
y_val   = diesel_val['PREÇO MÉDIO REVENDA']
y_test  = diesel_test['PREÇO MÉDIO REVENDA']

### Creating the baseline
The very first model predicts new values being equals to the last ones.
Let's create a new column containing predictions created in this way

In [10]:
# Each *ESTADO* has a value for date line.
# shift the column by one 
baseline_train = diesel_train.groupby(['ESTADO'])['PREÇO MÉDIO REVENDA'].shift(1)
baseline_val   = diesel_val.groupby(['ESTADO'])['PREÇO MÉDIO REVENDA'].shift(1)

Let's add this new column to the datasets

In [11]:
diesel_train['BASELINE'] = baseline_train
diesel_val['BASELINE']   = baseline_val

C:\Users\vipac\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\vipac\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Evaluating model performance
Metric: Mean squared logarithmic error

This metric evaluates the percentage error: "This metric is best to use when targets having exponential growth, such as population counts, average sales of a commodity over a span of years etc. Note that this metric penalizes an under-predicted estimate greater than an over-predicted estimate."
\begin{equation}
\frac{1}{N_{samples}} \sum_{i=0}^{N_{samples}-1} \left(Ln(1+y_{true}) - Ln(1+y_{pred})\right)^2
\end{equation}

In [12]:
baseline_val.notnull().shape, y_val[baseline_val.notnull()].shape, y_val.shape

((9315,), (9288,), (9315,))

In [13]:
from sklearn.metrics import mean_squared_log_error

idx_notnull = baseline_val.notnull() # taking only indexes with predictions not null

baseline_error = np.sqrt(mean_squared_log_error(
    y_val[idx_notnull], baseline_val[idx_notnull] )) # Root mean squared logarithmic error 
print('baseline_error:', baseline_error*100)

baseline_error: 0.66461058218652
